## Falcon 40B Model Deployment

In [4]:
# install supported sagemaker SDK
!pip install "sagemaker==2.163.0" --upgrade --quiet

In [ ]:
# !pip install accelerate

In [20]:
import json
import sagemaker
import boto3
import os
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

In [27]:
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1' # ml.g5.xlarge, ml.g5.48xlarge
# os.environ['AWS_DEFAULT_REGION'] = 'us-west-2' # ml.g5.xlarge, ml.g5.2xlarge, ml.g5.4xlarge, ml.g5.8xlarge

In [28]:
region = boto3.Session().region_name
region

'us-east-1'

In [32]:
try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'tiiuae/falcon-40b-instruct', 
	'SM_NUM_GPUS': json.dumps(4),
    # 'HF_MODEL_QUANTIZE': "bitsandbytes"
}

endpoint_name = 'sm-contract-audit'

In [33]:
# help(HuggingFaceModel)

In [34]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="0.8.2"),
	env=hub,
	role=role, 
)

In [35]:
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.48xlarge",  
	container_startup_health_check_timeout=300,
    endpoint_name=endpoint_name
  )

-----------------!

In [48]:
# predictor.predict({
# 	"inputs": "My name is Julien and I like to  ",
# })

In [37]:
def query_endpoint(payload):
    """Query endpoint and print the response"""
    response = predictor.predict(payload)
    #print(f"\033[1m Input:\033[0m {payload['inputs']}")
    print(f"\033[1m Output:\033[0m {response[0]['generated_text']}")

## Prompt Engineering

In [38]:
code = """
 // SPDX-License-Identifier: MIT
 pragma solidity ^0.8.17;

 contract CrowdFunding {
     struct Campaign {
         address owner;
         string title;
         string description;
         uint256 target;
         uint256 deadline;
         uint256 amountCollected;
         string image;
         address[] donators;
         uint256[] donations;
     }

     mapping(uint256 => Campaign) public campaigns;

     uint256 public numberOfCampaigns = 0;

     function createCampaign(
         address _owner,
         string memory _title,
         string memory _description,
         uint256 _target,
         uint256 _deadline,
         string memory _image
     ) public returns (uint256) {
         Campaign storage campaign = campaigns[numberOfCampaigns];

         require(
             campaign.deadline < block.timestamp,
             "The deadline should be a date in the future."
         );

         campaign.owner = _owner;
         campaign.title = _title;
         campaign.description = _description;
         campaign.target = _target;
         campaign.deadline = _deadline;
         campaign.amountCollected = 0;
         campaign.image = _image;

         numberOfCampaigns++;

         return numberOfCampaigns - 1;
     }

     function donateToCampaign(uint256 _id) public payable {
         uint256 amount = msg.value;

         Campaign storage campaign = campaigns[_id];

         campaign.donators.push(msg.sender);
         campaign.donations.push(amount);

         campaign.amountCollected = campaign.amountCollected + amount;
     }

     function withdraw(uint256 _id) public {
         Campaign storage campaign = campaigns[_id];

         require(
             campaign.owner == msg.sender,
             "Only the owner of the campaign can withdraw the funds."
         );

         require(
             campaign.amountCollected >= campaign.target,
             "The campaign has not reached its target yet."
         );

         require(
             campaign.deadline < block.timestamp,
             "The deadline has not passed yet."
         );

         (bool sent, ) = payable(campaign.owner).call{
             value: campaign.amountCollected
         }("");

         if (sent) {
             campaign.amountCollected = 0;
         }
     }

     function getCampaign(uint256 id)
         public
         view
         returns (
             address,
             string memory,
             string memory,
             uint256,
             uint256,
             uint256,
             string memory
         )
     {
         Campaign memory campaign = campaigns[id];
         return (
             campaign.owner,
             campaign.title,
             campaign.description,
             campaign.target,
             campaign.deadline,
             campaign.amountCollected,
             campaign.image
         );
     }
 }
"""

In [40]:
prompt = f"""You are an helpful Assistant, called Falcon and you know about Smart Conctracts.

User: Can you list the security vulnerability in the following smart contract code?

{code}

Falcon:"""

In [45]:
# define payload

# hyperparameters for llm
payload = {
  "inputs": prompt,
  "parameters": {
    "do_sample": True,
    "top_p": 0.9,
    "temperature": 0.9,
    #"max_new_tokens": 1024,
    #"repetition_penalty": 1.03,
    "stop": ["\nUser:","<|endoftext|>","</s>"]
  }
}

# send request to endpoint
response = predictor.predict(payload)

# print assistant respond
assistant = response[0]["generated_text"][len(prompt):]
print(assistant)

 The vulnerability in this smart contract code is that the function withdraw() does not take into consideration the possibility


In [46]:
# query_endpoint(payload)

In [47]:
new_prompt = f"""{prompt}{assistant}
User: How would you recommend to improve the code?
Falcon:"""
# update payload
payload["inputs"] = new_prompt

# send request to endpoint
response = predictor.predict(payload)

# print assistant respond
new_assistant = response[0]["generated_text"][len(new_prompt):]
print(new_assistant)

 The withdraw function should be modified to check if the campaign has reached its target before allowing the owner to


In [ ]:
# query_endpoint(payload):

In [ ]:
## Delete Endpoint

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()